In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline


import numpy as np
import haiku as hk
import jax
import jax.numpy as jnp
from functools import partial
from sbi_lens.normflow.models import AffineCoupling,  ConditionalRealNVP 
from tensorflow_probability.substrates import jax as tfp
tfd = tfp.distributions
import pickle

'unset XLA_FLAGS'


Populating the interactive namespace from numpy and matplotlib


/linkhome/rech/genmfd01/ulm75uc/.local/lib/python3.10/site-packages/haiku/_src/data_structures.py:37: FutureWarning: jax.tree_structure is deprecated, and will be removed in a future release. Use jax.tree_util.tree_structure instead.
  PyTreeDef = type(jax.tree_structure(None))
2023-07-04 15:02:15.344423: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/gpfslocalsup/pub/anaconda-py3/2021.05/envs/python-3.10.4/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


'unset XLA_FLAGS'

## Load data

In [2]:
#####################################################################################################################################################
with open("/gpfsdswork/projects/rech/ykz/ulm75uc/VMIM-vs-MSE-/data/sample_nd_mse.pkl", 'rb') as g:
    sample_nd_mse = pickle.load(g)
with open("/gpfsdswork/projects/rech/ykz/ulm75uc/VMIM-vs-MSE-/data/sample_nd_mae.pkl", 'rb') as g:
    sample_nd_mae = pickle.load(g)
with open("/gpfsdswork/projects/rech/ykz/ulm75uc/VMIM-vs-MSE-/data/sample_nd_vmim.pkl", 'rb') as g:
    sample_nd_vmim = pickle.load(g)
#####################################################################################################################################################
with open("/gpfsdswork/projects/rech/ykz/ulm75uc/VMIM-vs-MSE-/data/params_nd_mse.pkl", 'rb') as g:
    params_nd_mse = pickle.load(g)
with open("/gpfsdswork/projects/rech/ykz/ulm75uc/VMIM-vs-MSE-/data/params_nd_mae.pkl", 'rb') as g:
    params_nd_mae = pickle.load(g)
with open("/gpfsdswork/projects/rech/ykz/ulm75uc/VMIM-vs-MSE-/data/params_nd_vmim.pkl", 'rb') as g:
    params_nd_vmim = pickle.load(g)
#####################################################################################################################################################
y_mse= np.load("/linkhome/rech/genmfd01/ulm75uc/VMIM-vs-MSE-/data/compressed_data/COMPRESSED_fiducial_mse_year_10_without_noise_score_density.npy")
y_mae= np.load("/linkhome/rech/genmfd01/ulm75uc/VMIM-vs-MSE-/data/compressed_data/COMPRESSED_fiducial_mae_year_10_without_noise_score_density.npy")
y_vmim= np.load("/linkhome/rech/genmfd01/ulm75uc/VMIM-vs-MSE-/data/compressed_data/COMPRESSED_fiducial_vmim_year_10_without_noise_score_density.npy")
#####################################################################################################################################################

/linkhome/rech/genmfd01/ulm75uc/.local/lib/python3.10/site-packages/haiku/_src/data_structures.py:206: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, structure = jax.tree_flatten(mapping)


## Build the Normalizing flow

In [3]:
bijector_layers = [128] * 2

bijector= partial(
  AffineCoupling,
  layers=bijector_layers,
  activation=jax.nn.silu
)

NF= partial(
  ConditionalRealNVP,
  n_layers=4,
  bijector_fn=bijector
)

class Flow_nd(hk.Module):
    def __call__(self, y):
        nvp = NF(6)(y)
        return nvp


## Define the Kullback–Leibler divergence

In [4]:
def kl_divergence(distribution_1,distribution_2,y_1, y_2, sample): #sample are the values sampled from the distribution 1
    nvp_nd=  hk.without_apply_rng(
    hk.transform(
    jax.vmap(
      lambda theta, y : Flow_nd()(y).log_prob(theta).squeeze(),
      in_axes=(0,None))))
    logposterior_1=nvp_nd.apply(distribution_1, sample, y_1[0])
    logposterior_2= nvp_nd.apply(distribution_2, sample, y_2[0])
    return jnp.mean(logposterior_1)-jnp.mean(logposterior_2)   

In [5]:
vmim_mse_kl_div=kl_divergence(params_nd_vmim,params_nd_mse, y_vmim, y_mse,  sample_nd_vmim)
vmim_mae_kl_div=kl_divergence(params_nd_vmim,params_nd_mae, y_vmim, y_mae,  sample_nd_vmim)
mae_mse_kl_div=kl_divergence(params_nd_mae,params_nd_mse, y_mae, y_mse,  sample_nd_mae)

In [6]:
mse_mae_kl_div=kl_divergence(params_nd_mse,params_nd_mae, y_mse, y_mae,  sample_nd_mse)
mae_vmim_kl_div=kl_divergence(params_nd_mae,params_nd_vmim,y_mae, y_vmim, sample_nd_mae)
mse_vmim_kl_div=kl_divergence(params_nd_mse,params_nd_vmim,y_mse, y_vmim, sample_nd_mse)

In [7]:
def kl_divergence_prior(distribution,y_1, sample): #sample are the values sampled from the distribution 1
    nvp_nd=  hk.without_apply_rng(
    hk.transform(
    jax.vmap(
      lambda theta, y : Flow_nd()(y).log_prob(theta).squeeze(),
      in_axes=(0,None))))
    logposterior=nvp_nd.apply(distribution, sample, y_1[0])
    logprior= tfd.MultivariateNormalDiag(jnp.array([0.2664,0.0492,0.831,0.6727,0.9645,-1.0]), jnp.array([0.2,0.006,0.14,0.063,0.08,0.9])*jnp.ones(6)).log_prob(sample)
    return jnp.mean(logposterior)-jnp.mean(logprior)   

In [8]:
kl_vmim=kl_divergence_prior(params_nd_vmim, y_vmim,sample_nd_vmim)
kl_mse=kl_divergence_prior(params_nd_mse, y_mse,sample_nd_mse)
kl_mae=kl_divergence_prior(params_nd_mae, y_mae,sample_nd_mae)

In [9]:
kl_vmim

Array(6.1196303, dtype=float32)

In [10]:
kl_mse

Array(5.2431912, dtype=float32)

In [11]:
kl_mae

Array(5.5617747, dtype=float32)